Importing cellular automata & optimization classes, and other stuff

In [1]:
#Attempt 2 code.
#%pip install tables

import os
import sys
import shutil

from typing import List, Type, Callable, Dict
from numpy import int32
from numpy._typing import NDArray
import importlib

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

#from cax_sica.blender import Lattice, clear_initial
from cax_sica.genetic import Optimizer, Mutator, RulesetMutator, ArbitraryRulesetMutator, MutationSet
from cax_sica.objectives import surface_to_vol

import numpy as np
import pandas as pd

import time

Setting up optimizer and data logging code

In [ ]:
def log_mutation(data_list: List[Dict], mutations: List[MutationSet], objective_val: float):
    """
    Given the data list reference, the mutation set, and the objective value after applying it, add it to the data logging list
    """
    ic_cell_pos = []
    ic_state_old = []
    ic_state_new = []
    srt_cell_pos = []
    srt_state_old = []
    srt_state_new = []

    for ic_mut in mutations.ic_mutations:
        ic_cell_pos.append(ic_mut.cell_pos)
        ic_state_old.append(ic_mut.old_state)
        ic_state_new.append(ic_mut.new_state)
    for srt_mut in mutations.srt_mutations:
        srt_cell_pos.append(srt_mut.cell_pos)
        srt_state_old.append(srt_mut.old_state)
        srt_state_new.append(srt_mut.new_state)
    
    data_list.append({
        "ic_cell_pos": np.array(ic_cell_pos), 
        "ic_state_old": np.array(ic_state_old), 
        "ic_state_new": np.array(ic_state_new), 
        "srt_cell_pos": np.array(srt_cell_pos), 
        "srt_state_old": np.array(srt_state_old), 
        "srt_state_new": np.array(srt_state_new), 
        "objective": objective_val,
    })

def run_experiment(iters: int, grid_sz: int, ruleset_mutator_class: Type[Mutator], rule_set: List[NDArray], opt_func: Callable[[NDArray[int32]], int],
                   srt_num_mutate: int, ic_num_mutate: int, rule_mutate_prob: float):
    """
    Runs an experiment with the below hyperparameters:

    :param iters: The number of iterations the mutation algorithm (updating both IC and SRT) is going to run for
    :param grid_sz: The size of the square grid that we're going to update each iteration
    :param ruleset_mutator_class: The class of the SRT mutator we're going to use for this experiment
    :param rule_set: The set of rules that the SRT initially has (picked at random for each cell, then scrambled by the mutator)
    :param opt_func: The functions that gives the performance metric we're going to optimize
    :param srt_num_mutate: The number of SRT cells for which we're going to mutate the rule applied, each iteration
    :param ic_num_mutate: The number of IC cells for which we're going to mutate the rule applied, each iteration
    :param rule_mutate_prob: The probability, for each neighbor state tensor of the rule of a cell that's selected to be mutated, the final state is mutated
    """
    # TODO: separate SRT and IC mutations to have a certain number of each
    # TODO: add a flag to enable doing only SRT or only IC mutations in an iteration (in optimizer step, and then propagate into mutator)
    ruleset_mutator = ruleset_mutator_class(rules=rule_set, grid_size=grid_sz, mutate_p=1/(grid_sz**2) * (srt_num_mutate+ic_num_mutate), rule_mutate_p=rule_mutate_prob)

    optim = Optimizer(mutator=ruleset_mutator, objective=lambda grid: opt_func(grid))
    """
    Pandas Dataframe used to log experiment data is:

    ic_cell_pos (np.array) | ic_state_old (np.array) | ic_state_new (np.array) | srt_cell_pos (np.array) | srt_state_old (np.array) | srt_state_new (np.array) | objective (float)
    
    etc.

    initial state for IC is in entry 0 in ic_state_old, and SRT is in entry 0 in srt_state_old

    ic and srt mutation cell positions and states can have an extra dimension in the beginning to indicate they are batch updates
    """

    init_state = optim.state
    
    data_list = [{"ic_cell_pos": grid_sz, 
                  "ic_state_old": init_state.initial, 
                  "ic_state_new": None, 
                  "srt_cell_pos": -1, 
                  "srt_state_old": init_state.rules, 
                  "srt_state_new": None, 
                  "objective": 0}]

    for it in range(iters):
        if (it%5 == 0):
            print(f"Iteration {it}")
        # print(f"On iteration {it+1}...")
        accepted, new, old, mutations = optim.step()
        # data logging
        log_mutation(data_list, mutations, optim.objvalue)
        # if accepted:
        #     print("Got a better state!", optim.objvalue)

    # print(data_list)
    df = pd.DataFrame(data_list)
    # print(df)
    return df

timelogs = []

def repeat_experiment(experiment_name: str, num_expers: int, *args):
    """
    Perform (sequentially) multiple experiments that return a Pandas DataFrame and save all the data

    :param experiment_name: The name of the experiment to save the file
    :param num_expers: Number of times to run the experiment (and save all the data in one file)
    :param *args: The arguments to be passed to the experiment function
    """
    for i in range(num_expers):
        init = time.time()
        print(f'REPETITION {i}')
        ret_data = run_experiment(*args)
        timelogs.append(time.time() - init)
        print(f"Finished rep {i} in {time.time() - init}s")
        ret_data.to_hdf(f'data/{experiment_name}_{i}.h5', key='data', mode='a')


Setting up experiments and gathering data

In [3]:
ITERATIONS_SET = [100]
GRID_SIZE_SET = [32]
NUM_REPEAT = 50
EXPERIMENT_NAME = "test_experiment"

for iters in ITERATIONS_SET:
    for grid_sz in GRID_SIZE_SET:
        print(f"RUNNING EXPERIMENT {EXPERIMENT_NAME} WITH {iters} ITERATIONS AND {grid_sz} SIZE GRID")
        repeat_experiment(f"{EXPERIMENT_NAME}_{iters}ITERS_{grid_sz}GRID", NUM_REPEAT, iters, grid_sz, ArbitraryRulesetMutator, [[0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0],[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], surface_to_vol, 10, 10, 2/3)

RUNNING EXPERIMENT test_experiment WITH 100 ITERATIONS AND 32 SIZE GRID
REPETITION 0
Iteration 0
Init time: 0.1362311840057373
Operation time: 0.16826486587524414
Total time: 0.16826486587524414
Init time: 0.16198277473449707
Operation time: 0.1809830665588379
Total time: 0.18198585510253906
Init time: 0.13726544380187988
Operation time: 0.1562366485595703
Total time: 0.15723156929016113
Init time: 0.1651761531829834
Operation time: 0.1842503547668457
Total time: 0.1852552890777588
Init time: 0.14157819747924805
Operation time: 0.15854454040527344
Total time: 0.15959596633911133
Iteration 5
Init time: 0.15195369720458984
Operation time: 0.17095136642456055
Total time: 0.17095136642456055
Init time: 0.1346578598022461
Operation time: 0.15223336219787598
Total time: 0.15323448181152344
Init time: 0.16163015365600586
Operation time: 0.17962336540222168
Total time: 0.18062281608581543
Init time: 0.14017152786254883
Operation time: 0.15717625617980957
Total time: 0.15819096565246582
Init ti

KeyboardInterrupt: 

In [ ]:
print(timelogs)

[18.081819772720337, 16.409464836120605, 16.46668815612793, 16.149325370788574, 16.261744499206543, 16.524474382400513, 16.581706762313843, 16.684239387512207, 16.408851385116577, 16.45926284790039, 16.07528805732727, 16.32613229751587, 16.32884931564331, 16.15177607536316, 16.243895292282104, 16.18384575843811, 16.12856364250183, 16.325840950012207, 16.173898220062256, 16.33220911026001, 16.378711700439453, 16.14823055267334, 16.25066828727722, 16.122880697250366, 16.299094676971436, 16.22087812423706, 16.244765281677246, 16.235554456710815, 16.151467084884644, 16.195945262908936, 16.291621923446655, 16.10358500480652, 16.279123544692993, 16.106332302093506, 16.23129439353943, 16.149306535720825, 16.404770851135254, 16.23179054260254, 16.398053646087646, 16.482661485671997, 16.352272033691406, 16.330648183822632, 16.18097186088562, 16.290521144866943, 16.160972595214844, 16.30544877052307, 16.25381565093994, 16.268326997756958, 16.497699737548828, 16.280954599380493]
